Orodja

In [1]:
import csv
import os
import requests
import sys
import re


def pripravi_imenik(ime_datoteke):
    '''Če še ne obstaja, pripravi prazen imenik za dano datoteko.'''
    imenik = os.path.dirname(ime_datoteke)
    if imenik:
        os.makedirs(imenik, exist_ok=True)


def shrani(url, ime_datoteke, vsili_prenos=False):
    '''Vsebino strani na danem naslovu shrani v datoteko z danim imenom.'''
    try:
        print('Shranjujem {}...'.format(url), end='')
        sys.stdout.flush()
        if os.path.isfile(ime_datoteke) and not vsili_prenos:
            print('shranjeno že od prej!')
            return
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        print('stran ne obstaja!')
    else:
        pripravi_imenik(ime_datoteke)
        with open(ime_datoteke, 'w') as datoteka:
            datoteka.write(r.text)
            print('shranjeno!')


def vsebina_datoteke(ime_datoteke):
    '''Vrne niz z vsebino datoteke z danim imenom.'''
    with open(ime_datoteke) as datoteka:
        vsebina = datoteka.read()
    return vsebina


def datoteke(imenik):
    '''Vrne imena vseh datotek v danem imeniku skupaj z imenom imenika.'''
    return [os.path.join(imenik, datoteka) for datoteka in os.listdir(imenik)]


def zapisi_tabelo(slovarji, imena_polj, ime_datoteke):
    '''Iz seznama slovarjev ustvari CSV datoteko z glavo.'''
    pripravi_imenik(ime_datoteke)
    with open(ime_datoteke, 'w') as csv_dat:
        writer = csv.DictWriter(csv_dat, fieldnames=imena_polj)
        writer.writeheader()
        for slovar in slovarji:
            writer.writerow(slovar)

Funkcija, ki z regularnimi izrazi izlušči podatke iz html kode

-Regularni izraz, ki najde: povezavo, ime, državo in točke posemezne borke/ca

In [2]:
regex_judo = re.compile(
        r'<a href="/athlete/(?P<id>\d+)">\s*(?P<ime>.*?\s.*?)\s*</a>'  
        r'.*?<img src="https:.*?<small>(?P<drzava>.*?)</small>'
        r'.*?<td class="text-center">\s*(?P<tocke>\d+) points\s*</td>'
        ,flags=re.DOTALL)

In [4]:
def pocisti_ime(judoka):
    podatki = judoka.groupdict()
    podatki['ime'].capitalize()
    podatki['ime'] = re.sub('\s+',' ',podatki['ime'])
        
    return podatki

def pripravi_db():   
    judoke = []
    for dat in datoteke('JUDO/'):
        
        for judoka in re.finditer(regex_judo, vsebina_datoteke(dat)):

            #print (pocisti_ime(judoka))
            judoke.append(pocisti_ime(judoka))

    #zapisi_tabelo(judoke, ['id','ime', 'drzava', 'tocke'], 'judoke.csv')
    
    #podfunkicja, ki prenese še html datoteko s podatki o številu olimpijskih medalj posamezne borke/ca
    for j in judoke:
        naslov = 'https://www.ijf.org/athlete/{}/results'.format(j['id'])
        datoteka = 'JUDOKE/{}.html'.format(j['ime'])
        shrani(naslov, datoteka)
        
        
    

    
pripravi_db()

Shranjujem https://www.ijf.org/athlete/385/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/11651/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/3278/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/3393/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/3394/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/14698/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/245/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/2618/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/2505/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/1982/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/2620/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/3740/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/2378/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/7214/results...shranjeno!
Shranjujem https://www.ijf.org/athlete/712/results...shranjeno!
Shranjujem https://www.ijf

Regularni izraz, ki poišče število olimpijskih medalj o posemezni borki/cu po html datoteki 

In [2]:
regex_oseba = re.compile(
    r'<a href="/athlete/(?P<id>\d+)/results\?results_rank'
    r'.*?group=og&results_place=1">\s+?<div>(?P<zlato>\d+)</div>'
    r'.*?group=og&results_place=2">\s+?<div>(?P<srebro>\d+)</div>'
    r'.*?group=og&results_place=3">\s+?<div>(?P<bron>\d+)</div>'
    ,flags=re.DOTALL)

In [ ]:
def pripravi_osebo():
    oseba =[]
    for dat in datoteke('JUDOKE/'):
        
        for ujemanje in re.finditer(regex_oseba, vsebina_datoteke(dat)):
            oseba.append(ujemanje.groupdict())
            #print (ujemanje.groupdict())
        
    zapisi_tabelo(oseba, ['id','zlato', 'srebro', 'bron'], 'medalje.csv')
            
            
            
pripravi_osebo()